In [ ]:
#@title Define if we are on Colab and mount drive { display-mode: "form" }
run_params = {}
try:
  from google.colab import drive
  drive.mount('/content/gdrive')
  run_params['IN_COLAB'] = True
except:
  run_params['IN_COLAB'] = False

In [ ]:
#@title (COLAB ONLY) Clone GitHub repo { display-mode: "form" }
# %%capture
if run_params['IN_COLAB']:
  !git clone https://github.com/lluissalord/radiology_ai.git

  %cd radiology_ai

In [ ]:
#@title Setup environment and Colab general variables { display-mode: "form" }
# %%capture
%run colab_pip_setup.ipynb

In [ ]:
from train.params import default_params, SSL_params
run_params, cb_params, loss_params = default_params(in_colab=run_params['IN_COLAB'])

In [ ]:
#@title Move images from Drive to temporary folder here to be able to train models { display-mode: "form" }
# %%capture
%run move_raw_preprocess.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fastai.basics import (
    RocAuc,
    RocAucBinary,
    error_rate,
    BalancedAccuracy,
    F1Score,
    FBeta,
    Precision,
    Recall,
    ValueMetric,
    Learner,
)
from fastai.callback.all import *
from fastai.callback.tensorboard import TensorBoardCallback

from train.data.dataframe import generate_dfs
from train.data.dataloader import (
    AllLabelsInBatchDL,
    define_ds_params,
    define_dls_params,
    get_label_dl,
    get_unlabel_dls,
)
from train.losses.losses import define_losses
from train.models import get_training_model
from train.optimizer import get_optimizer
from train.self_supervised import get_self_supervised_model
from train.tfms import (
    get_item_tfms,
    get_batch_tfms
)
from train.utils import seed_everything

from train_script import train

In [ ]:
run_params["SSL"] = run_params["SSL_FIX_MATCH"]
run_params["SSL"] = None
run_params, cb_params, loss_params = SSL_params(run_params)
loss_params["use_SCL"] = True
# run_params["LAMBDA_U"] = 0
run_params["OPT_LR"] = 0.005

from fastai.vision import models
# run_params["MODEL"] = models.resnet152
# run_params["BATCH_SIZE"] = 32

run_params["POSITIVES_ON_TRAIN"] = 0.5
run_params["ALL_LABELS_IN_BATCH"] = False

run_params["OPTIMIZER"] = "AdaBelief"

In [ ]:
debug = True
run_params['SEED'] = 42

seed_everything(run_params['SEED'])

In [ ]:
# Data
label_df, unlabel_df = generate_dfs(run_params, debug=debug)
train_df = label_df[label_df['Dataset'] == 'train']

In [ ]:
# Item (CPU) Transformations
item_tfms = get_item_tfms(run_params)

# Batch (GPU) transformation
label_tfms, unlabel_tfms = get_batch_tfms(run_params)

In [ ]:
# Self supervised
model_self_sup = None
if run_params['SELF_SUPERVISED']:
    if run_params['IN_COLAB']:
        # Load the TensorBoard notebook extension
        %load_ext tensorboard
        %tensorboard --logdir {'"' + os.path.join(run_params['PATH_PREFIX'] , 'tb_logs', 'simCLR') + '"'}

    model_self_sup = get_self_supervised_model(run_params)

In [ ]:
## Define Dataset parameters
label_ds_params, unlabel_ds_params = define_ds_params(
    run_params, item_tfms, label_tfms
)

## Define DataLoaders parameters
dls_params, unlabel_dls_params = define_dls_params(run_params)

# DataLoaders
if debug:
    print(f"==> Preparing label dataloaders")
label_dl = get_label_dl(run_params, dls_params, label_ds_params, label_df)

unlabel_dls = [None]
if run_params["SSL"] and run_params["LAMBDA_U"] != 0:
    if debug:
        print(f"==> Preparing unlabel dataloaders")
    unlabel_dls = get_unlabel_dls(
        run_params, unlabel_tfms, unlabel_dls_params, unlabel_ds_params, unlabel_df
    )

In [ ]:
# Callbacks
if debug:
    print(f'==> Preparing callbacks')

if run_params["SSL"] == run_params["SSL_FIX_MATCH"]:
    from semisupervised.fixmatch.callback import FixMatchCallback as SSLCallback
elif run_params["SSL"] == run_params["SSL_MIX_MATCH"]:
    from semisupervised.mixmatch.callback import MixMatchCallback as SSLCallback

cbs = []
# cbs = [
#     TensorBoardCallback(
#             log_dir=os.path.join(
#                 run_params['PATH_PREFIX'] ,
#                 'tb_logs',
#                 'main',
#                 run_params['MODEL_SAVE_NAME']
#             ),
#             projector=True,
#     )
# ]
if not run_params["SSL"]:
    cbs.insert(0, MixUp())
elif run_params["LAMBDA_U"] != 0:
    ssl_cb = SSLCallback(*unlabel_dls, **cb_params)
    cbs.append(ssl_cb)

    if run_params["SSL"] == run_params["SSL_MIX_MATCH"]:
        cbs.append(MixUp(alpha=run_params["ALPHA"]))

if run_params["GRAD_MAX_NORM"] is not None:
    cbs.append(GradientClip(max_norm=run_params["GRAD_MAX_NORM"]))

# Debugging Callback to print out some values of the data being proceess on each batch
# cbs.append(DebuggingCallback(start_epoch=0))

cbs.append(
    SaveModelCallback(
        # monitor="valid_fbeta_score",
        # comp=np.greater,
        min_delta=0,
        reset_on_fit=False,
    )
)

cbs.append(
    ReduceLROnPlateau(
        # monitor="valid_fbeta_score",
        # comp=np.greater,
        min_delta=0,
        reset_on_fit=True,
        patience=5,
        factor=5.,
        # min_lr=run_params["OPT_LR"] * 1E-4
    )
)

In [ ]:
# Optimizer
opt_func, opt_params = get_optimizer(run_params)

In [ ]:
# Model
if debug:
    print("==> Creating model")
model, splitter = get_training_model(
    run_params,
    loss_params,
    train_df,
    n_in=label_dl.n_inp,
    model_self_sup=model_self_sup,
)

In [ ]:
# Loss
if debug:
    print("==> Defining loss")
loss_func = define_losses(run_params, loss_params, train_df, unlabel_dls)

In [ ]:
import mlflow

exp_name = 'nb-run'
experiment_id = mlflow.set_experiment(exp_name)

os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])

# MLFlow not compatible with TensorBoard
cbs = [cb for cb in cbs if cb.name != 'tensor_board']

In [ ]:
# Learner
if debug:
    print("==> Defining learner")
# Adapt metrics depending on the number of labels
if label_dl.c == 2:
    average = "binary"
    roc_auc = RocAucBinary()
else:
    average = "macro"
    roc_auc = RocAuc()

metrics = [
    error_rate,
    BalancedAccuracy(),
    roc_auc,
    # FBeta(0.5, average=average),
    F1Score(average=average),
    FBeta(2, average=average),
    Precision(average=average),
    Recall(average=average),
]

if run_params["SSL"] and run_params["LAMBDA_U"] != 0:
    # metrics.insert(0, ValueMetric(loss_func.total_loss))
    metrics.insert(0, ValueMetric(loss_func.Lu))
    metrics.insert(0, ValueMetric(loss_func.w))
    metrics.insert(0, ValueMetric(loss_func.Lx))

learn = Learner(
    label_dl,
    model,
    splitter=splitter,
    loss_func=loss_func,
    opt_func=opt_func,
    **opt_params,
    lr=run_params["OPT_LR"],
    metrics=metrics,
    cbs=cbs,
)
learn.recorder.train_metrics = True

learn.to_fp16();

In [ ]:
if learn.opt is None: learn.create_opt()
learn.opt.set_hyper('lr', learn.lr if run_params["OPT_LR"] is None else run_params["OPT_LR"])
learn.freeze()
learn.fit(run_params["TRAIN_FREEZE_EPOCHS"], lr=slice(run_params["OPT_LR"]))

mlflow.fastai.autolog()

# Start mlflow run
with mlflow.start_run(experiment_id=experiment_id):

    mlflow.log_params(run_params)

    # learn.to_fp16();
    learn.unfreeze()
    learn.fit(30, lr=slice(run_params["OPT_LR"]/5/100, run_params["OPT_LR"]/5))
    learn.to_fp32();

In [ ]:
run_params["SAVE_MODEL"] = True

In [ ]:
if run_params['SAVE_MODEL']:

    from fastai.learner import save_model

    if not os.path.exists(run_params['MODELS_FOLDER']):
        os.makedirs(run_params['MODELS_FOLDER'])

    save_model(file=run_params['MODEL_SAVE_PATH'], model=learn.model, opt=learn.opt)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(
    np.array(learn.recorder.values)[
        :, learn.recorder.metric_names.index("valid_fbeta_score") - 1
    ]
)
plt.plot(
    np.array(learn.recorder.values)[
        :, learn.recorder.metric_names.index("train_fbeta_score") - 1
    ]
)
plt.ylabel("fbeta_score")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(
    np.array(learn.recorder.values)[
        :, learn.recorder.metric_names.index("valid_loss") - 1
    ]
)
plt.plot(
    np.array(learn.recorder.values)[
        :, learn.recorder.metric_names.index("train_loss") - 1
    ]
)
plt.ylabel("loss")
plt.yscale("log")
plt.show()

In [ ]:
# Start mlflow run
with mlflow.start_run(experiment_id=experiment_id):

    mlflow.log_params(run_params)

    # Review use of EarlyStopping with fine_tune
    # learn.fit(1)
    # Not able to use MLFlow autolog with repeat cycles
    i = 0
    while i < run_params["REPEAT_ONE_CYCLE"]:
        learn.fine_tune(
            run_params["TRAIN_EPOCHS"],
            run_params["OPT_LR"],
            freeze_epochs=run_params["TRAIN_FREEZE_EPOCHS"] if not i else 0,
        )
        plt.plot(
            np.array(learn.recorder.values)[
                :, learn.recorder.metric_names.index("valid_fbeta_score") - 1
            ]
        )
        plt.ylabel("valid_fbeta_score")
        plt.show()
        i += 1

        # This should be done on ParamSched way
        # run_params['OPT_LR'] /= 10
        # run_params['TRAIN_EPOCHS'] /= 2

    learn.to_fp32();

In [ ]:
class RecordMetricsExtraData(Callback):
    remove_on_fetch,order = True,Recorder.order+1

    def __init__(self, items, prefix='extra'):
        self.items = items
        self.prefix = prefix

    def before_fit(self):
        self.extra_dl = self.dls.test_dl(self.items, with_labels=True)
        names = L('loss') + self.metrics.attrgot('name')
        names = names.map(self.prefix + '_{}')
        index = -2 if self.recorder.add_time else -1
        if self.recorder.add_time:
            self.recorder.metric_names = self.recorder.metric_names[:-1] + names + [self.recorder.metric_names[-1]]
        else:
            self.recorder.metric_names += names
        self.metric_names_added = True

    def after_validate(self):
        if hasattr(self, "lr_finder") or hasattr(self, "gather_preds"):
            return
        
        preds, targs, losses = learn.get_preds(dl=self.extra_dl, with_loss=True)
        max_preds, outs = torch.max(preds, axis=1)

        print(preds)
        print(targs)
        print(outs)
        print(losses)
        values = [losses.mean()]
        for metric in self.metrics:
            if isinstance(metric, AvgMetric):
                metric = AccumMetric(metric.func)
            
            try:
                metric_value = metric(preds, targs)
            except AssertionError:
                metric_value = metric(outs, targs)
            
            values.append(metric_value)
        
        self.recorder.log += values

In [ ]:
# cbs.append(RecordMetricsExtraData(label_df[label_df['Dataset'] == 'valid']))
cbs[-1] = RecordMetricsExtraData(label_df[label_df['Dataset'] == 'valid'])

In [ ]:
learn.add_cb(RecordMetricsExtraData(label_df[label_df['Dataset'] == 'valid'][:16]))
learn.cbs[-1].before_fit()
learn.cbs[-1].after_validate()

In [ ]:
class EarlyStoppingCallback(TrackerCallback):
    "A `TrackerCallback` that terminates training when monitored quantity stops improving."
    def __init__(self, monitor='valid_loss', comp=None, min_delta=0., patience=1, reset_on_fit=True):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta, reset_on_fit=reset_on_fit)
        self.patience = patience

    def before_fit(self): 
        if self.reset_on_fit:
            self.wait = 0
        super().before_fit()
    def after_epoch(self):
        "Compare the value monitored to its best score and maybe stop training."
        super().after_epoch()
        if self.new_best: self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                print(f'No improvement since epoch {self.epoch-self.wait}: early stopping')
                raise CancelFitException()

In [ ]:
plt.plot(np.array(data)[:,learn.recorder.metric_names.index(monitor) - 1])
plt.plot(np.array(data)[:,learn.recorder.metric_names.index('train_fbeta_score') - 1])
plt.ylabel(monitor)
plt.show()

In [ ]:
plt.plot(np.array(data)[:,learn.recorder.metric_names.index('train_loss') - 1])
plt.plot(np.array(data)[:,learn.recorder.metric_names.index('valid_loss') - 1])
plt.ylabel(monitor)
plt.show()

In [ ]:
learn.freeze()
learn.lr_find()

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
if run_params['IN_COLAB']:
  # Load the TensorBoard notebook extension
  %load_ext tensorboard

  %tensorboard --logdir {'"' + os.path.join(run_params['PATH_PREFIX'] , 'tb_logs', 'main') + '"'}

In [ ]:
learn.fine_tune(100, run_params['LR'], freeze_epochs=3)

In [ ]:
plt.plot(learn.loss_func.losses['Lx'])
plt.show()

plt.plot(learn.loss_func.losses['Lu'])
plt.show()

In [ ]:
learn.fine_tune(100, run_params['LR'], freeze_epochs=0)

In [ ]:
# Select only the top K images with largest loss

from fastai.interpret import ClassificationInterpretation
# from fastai2_extensions.interpret.all import *
# from fastai_amalgam.interpret.all import *
from fastai.basics import first, ifnone

k = 9
largest = True
dls_idx = 1

preds, targs, decoded, all_losses = learn.get_preds(dls_idx, with_decoded=True, with_loss=True)
losses, idx = all_losses.topk(ifnone(k, len(all_losses)), largest=largest)

top_losses_dl = learn.dls.test_dl(learn.dls[dls_idx].items.iloc[idx])
top_losses_dl.bs = len(idx)

interp = ClassificationInterpretation(
    learn.dls[dls_idx],
    inputs=first(top_losses_dl),
    preds=preds[idx],
    targs=targs[idx],
    decoded=decoded[idx],
    losses=losses,
    # *tuple(map(lambda x: x[idx], learn.get_preds(dls_idx, with_input=True, with_loss=True, with_decoded=True)))
)
interp.plot_top_losses(k=k, cmap=plt.cm.bone)

In [ ]:
dls_idx = 2
dls_idx = 0

preds, targs, decoded, all_losses = learn.get_preds(dls_idx, with_decoded=True, with_loss=True)
max_preds, outs = torch.max(preds, axis=1)

for metric in metrics:
    try:
        metric_name = metric.name
    except AttributeError:
        metric_name = metric.__name__

    try:
        print(metric_name, ':', metric(preds, targs))
    except AssertionError:
        print(metric_name, ':', metric(outs, targs))

In [ ]:
# Plot GradCAM for the top K images with largest loss

from fastai_amalgam.interpret.gradcam import gradcam

dls_idx = 1

for i in idx:
    gcam = gradcam(learn, learn.dls[dls_idx].items.iloc[i.numpy()]['Raw_preprocess'], labels=['0', '1'], show_original=True, cmap=plt.cm.bone)
    display(gcam)
    print()

In [ ]:
# Plot GradCAM for the true positive images

from fastai_amalgam.interpret.gradcam import gradcam

dls_idx = 0
label_idxs = learn.dls[dls_idx].items[learn.dls[dls_idx].items['Target'] != '0'].index

for i in label_idxs:
    gcam = learn.gradcam(learn.dls[dls_idx].items.loc[i, 'Raw_preprocess'], labels=['0', '1'], show_original=True, cmap=plt.cm.bone)
    display(gcam)
    print()

In [ ]:
# Plot GradCAM for the true positive images

from fastai_amalgam.interpret.gradcam import gradcam

dls_idx = 1
label_idxs = learn.dls[dls_idx].items[learn.dls[dls_idx].items['Target'] != '0'].index

for i in label_idxs:
    gcam = learn.gradcam(learn.dls[dls_idx].items.loc[i, 'Raw_preprocess'], labels=['0', '1'], show_original=True, cmap=plt.cm.bone)
    display(gcam)
    print()

In [ ]:
# Plot GradCAM for the true positive images

from fastai_amalgam.interpret.gradcam import gradcam

dls_idx = 2
label_idxs = learn.dls[dls_idx].items[learn.dls[dls_idx].items['Target'] != '0'].index

for i in label_idxs:
    gcam = learn.gradcam(learn.dls[dls_idx].items.loc[i, 'Raw_preprocess'], labels=['0', '1'], show_original=True, cmap=plt.cm.bone)
    display(gcam)
    print()